In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
--Criando as tabelas principais através do repositório
DROP TABLE IF EXISTS Depressao;
DROP TABLE IF EXISTS MentalFacilities;

CREATE TABLE Depressao(
    nomePais VARCHAR(52),
    anoPesquisa varchar(16),
    populacao bigint,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
) AS SELECT
    Entity,
    Year, 
    Population,
    Prevalence_Depressive_Disorders_Male,
    Prevalence_Depressive_Disorders_Female
FROM CSVREAD('../data/depression_mh.csv');

CREATE TABLE MentalFacilities(
    nomePais VARCHAR(52),
    anoPesquisa VARCHAR(16),
    hospitais FLOAT,
    servicos_de_saude FLOAT,
    servicos_de_saude_nao_gov FLOAT,
    servicos_de_saude_day_threat FLOAT,
    comunidade_residencial FLOAT
) AS SELECT
    Country,
    Year, 
    Mental_hospitals,
    Mental_health_units,
    Mental_health_outpatient,
    Mental_health_day_treatment,
    Community_residential_facilities
FROM CSVREAD('../data/mental_health_facilities_gho.csv');

In [3]:
--Normalizando os dados da tabela de Depressão
DELETE FROM Depressao D WHERE D.porcentagemHomens IS NULL;
--SELECT * FROM Depressao;

In [4]:
--Nrmalizando os dados da tabela de MentalFacilities
UPDATE MentalFacilities SET hospitais = 0 WHERE hospitais IS NULL;
UPDATE MentalFacilities SET servicos_de_saude = 0 WHERE servicos_de_saude IS NULL;
UPDATE MentalFacilities SET servicos_de_saude_nao_gov = 0 WHERE servicos_de_saude_nao_gov IS NULL;
UPDATE MentalFacilities SET servicos_de_saude_day_threat = 0 WHERE servicos_de_saude_day_threat IS NULL;
UPDATE MentalFacilities SET comunidade_residencial = 0 WHERE comunidade_residencial IS NULL;

SELECT * FROM MentalFacilities;

DROP TABLE IF EXISTS MentalFacilities_v2;
CREATE TABLE MentalFacilities_v2 (
    nomePais VARCHAR(52),
    anoPesquisa INT,
    porcentagem FLOAT
)

In [5]:
--Como esta tabela de MentaFacilities tinha muitos valores diferentes mas que no fundo representavam todos o número de prédios existentes no país
--(porcentagem em 100.000), decidi somar todos e colocar esta soma em um só valor com o nome de 'porcentagem' nesta nova tabela (MentaFacilities_v2)
INSERT INTO MentalFacilities_v2 (nomePais, anoPesquisa, porcentagem)
SELECT MF.nomePais, MF.anoPesquisa, (MF.hospitais + MF.servicos_de_saude + MF.servicos_de_saude_nao_gov + MF.servicos_de_saude_day_threat + MF.comunidade_residencial)
FROM MentalFacilities MF WHERE CAST(SUBSTRING(MF.anoPesquisa, 1, 4) AS INT) = MF.anoPesquisa;

In [6]:
SELECT * FROM MentalFacilities_v2;

In [6]:
--Fazendo a junção entre as tabelas
DROP TABLE IF EXISTS Juncao; 
CREATE TABLE Juncao (
    nomePais VARCHAR(53),
    anoPesquisa int,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
    porcentagemInstalacoes FLOAT
);

In [7]:
--Nesta query e na próxima faço uma pesquisa para ver qual a pesquisa mais recente do respectivo país, para que duas pesquisas de anos diferentes
--de um mesmo país não interfiram no resultado final
DROP VIEW IF EXISTS MAX_ANO;
CREATE VIEW MAX_ANO AS
SELECT nomePais, MAX(anoPesquisa) as ano_max FROM MentalFacilities_v2
GROUP BY nomePais, anoPesquisa;

--SELECT * FROM MAX_ANO;

In [8]:
INSERT INTO Juncao (nomePais, anoPesquisa, porcentagemHomens, porcentagemMulheres, porcentagemInstalacoes)
SELECT MF.nomePais, D.anoPesquisa, ROUND(D.porcentagemHomens , 4), ROUND(D.porcentagemMulheres, 4), ROUND(MF.porcentagem, 3)
FROM MentalFacilities_v2 MF, Depressao D, MAX_ANO MA
WHERE MF.nomePais = D.nomePais AND MF.nomePais = MA.nomePais AND MF.anoPesquisa = MA.ano_max AND D.anoPesquisa = MA.ano_max;
SELECT * FROM Juncao;

In [9]:
--Interessante constatação de que mulheres aparentemente relataram em maior número sofrerem de problemas mentais (quase 2x mais!)
SELECT ROUND(SUM(porcentagemHomens)/124 , 4) as homens, ROUND(SUM(porcentagemMulheres)/124 ,4) as mulheres FROM Juncao;

In [56]:
SELECT * FROM Juncao ORDER BY porcentagemInstalacoes DESC;

In [10]:
--Aqui, assim como foi feito em ConectandoDatabasesv3, criei um Caso01 para analisar o número de Instalações presentes nos países de acordo
--com a porcentagem de Homens e Mulheres com Depressão nos mesmos
DROP TABLE IF EXISTS Caso01;
CREATE TABLE Caso01(
    nomeRange VARCHAR(52),
    porcentagemDMedia DOUBLE,
    porcentagemIMedia DOUBLE
);

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes > 8;
UPDATE Caso01 SET nomeRange = 'High Facilities Percentage' WHERE porcentagemIMedia > 8;

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes < 8 AND J.porcentagemInstalacoes > 1.9;
UPDATE Caso01 SET nomeRange = 'Medium Facilities Percentage' WHERE porcentagemIMedia < 8 AND porcentagemIMedia > 1.9;

INSERT INTO Caso01 (porcentagemDMedia, porcentagemIMedia)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4)
FROM Juncao J WHERE J.porcentagemInstalacoes < 1.9;
UPDATE Caso01 SET nomeRange = 'Low Facilities Percentage' WHERE porcentagemIMedia < 1.9;

In [11]:
--Para minha surpresa, não se mostrou haver uma correlação muito forte
SELECT * FROM CASO01;

In [12]:
--Criada outra tabela com dados de IDH, através da tabela no repositório
DROP TABLE IF EXISTS IDH;

CREATE TABLE IDH(
    nomePais VARCHAR(52),
    IDH VARCHAR(15)
) AS SELECT
    pais, 
    paises_IDH
FROM CSVREAD('../data/paises_idh.csv');

DELETE FROM IDH WHERE nomePais = 'Hong Kong' OR nomePais = 'Palestine';

In [13]:
--Segunda tabela de Junção de dados: 
--relacionará todos os 3 valores: porcentagem de instalações, porcentagem de depressão e IDH dos respectivos países
DROP TABLE IF EXISTS Juncao2; 
CREATE TABLE Juncao2 (
    nomePais VARCHAR(53),
    anoPesquisa int,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
    porcentagemInstalacoes FLOAT,
    IDH FLOAT
);

In [14]:
--Populando esta tabela Junção 2
INSERT INTO Juncao2 (nomePais, anoPesquisa, porcentagemHomens, porcentagemMulheres, porcentagemInstalacoes, idh)
SELECT MF.nomePais, D.anoPesquisa, ROUND(D.porcentagemHomens , 4), ROUND(D.porcentagemMulheres, 4), ROUND(MF.porcentagem, 3), I.idh
FROM MentalFacilities_v2 MF, Depressao D, MAX_ANO MA, IDH I
WHERE MF.nomePais = D.nomePais AND MF.nomePais = MA.nomePais AND MF.anoPesquisa = MA.ano_max AND D.anoPesquisa = MA.ano_max AND I.nomePais = MF.nomePais
--SELECT * FROM Juncao2;

In [15]:
SELECT * FROM Juncao2;

In [16]:
--Análise destes 3 aspectos com relação à porcentagem de Instalações de cada país
DROP TABLE IF EXISTS Caso02;
CREATE TABLE Caso02(
    nomeRange VARCHAR(52),
    porcentagemDMedia DOUBLE,
    porcentagemIMedia DOUBLE,
    IDH FLOAT
);

INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes > 8;
UPDATE Caso02 SET nomeRange = 'High Facilities Percentage' WHERE porcentagemIMedia > 8;

INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes < 8 AND J.porcentagemInstalacoes > 1.9;
UPDATE Caso02 SET nomeRange = 'Medium Facilities Percentage' WHERE porcentagemIMedia < 8 AND porcentagemIMedia > 1.9;

INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes < 1.9;
UPDATE Caso02 SET nomeRange = 'Low Facilities Percentage' WHERE porcentagemIMedia < 1.9;

In [17]:
SELECT * FROM Caso02;

In [29]:
--Terceira tabela de Junção:
--Terá os valores de porcentagem de Depressão e Nome do Grupo(Comunidade) feito na análise de Grafo
DROP TABLE IF EXISTS Juncao3; 
CREATE TABLE Juncao3 (
    nomePais VARCHAR(53),
    anoPesquisa int,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
    nomeGrupo int
);

In [30]:
--Pegando a tabela com os Grupos/Comunidades do repositório
DROP TABLE IF EXISTS Dietas;
CREATE TABLE Dietas(
    nomePais VARCHAR(52) NOT NULL,
    nomeGrupo VARCHAR(50) NOT NULL,
) AS SELECT
    name,
    communityId
FROM CSVREAD('../data/comunidade_dietas.csv');

In [31]:
--Populando a tabela
INSERT INTO Juncao3 (nomePais, anoPesquisa, porcentagemHomens, porcentagemMulheres, nomeGrupo)
SELECT MF.nomePais, D.anoPesquisa, ROUND(D.porcentagemHomens , 4), ROUND(D.porcentagemMulheres, 4), Di.nomeGrupo
FROM MentalFacilities_v2 MF, Depressao D, MAX_ANO MA, Dietas Di
WHERE MF.nomePais = D.nomePais AND MF.nomePais = MA.nomePais AND MF.anoPesquisa = MA.ano_max AND D.anoPesquisa = MA.ano_max AND Di.nomePais = MF.nomePais
--SELECT * FROM Juncao2;

In [32]:
SELECT * FROM Juncao3;

In [33]:
--Tabela simples com apenas dois valores: grupo da comunidade e a porcentagem de Homens e Mulheres com Depressão
DROP TABLE IF EXISTS Caso03;
CREATE TABLE Caso03(
    nomeGrupo VARCHAR(52),
    porcentagemDMedia DOUBLE,
);

In [34]:
--Análise da média dos casos de Depressão entre os países de cada Grupo/Comunidade
SELECT nomeGrupo, ROUND(AVG(porcentagemHomens), 4) as depressao_homens, ROUND(AVG(porcentagemMulheres), 4) as depressao_mulheres
FROM Juncao3
GROUP BY nomeGrupo;

In [39]:
DROP TABLE IF EXISTS Caso02;
CREATE TABLE Caso02(
    nomeRange VARCHAR(52),
    porcentagemDMedia DOUBLE,
    porcentagemIMedia DOUBLE,
);

In [38]:
INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes > 8;
UPDATE Caso02 SET nomeRange = 'High Facilities Percentage' WHERE porcentagemIMedia > 8;

INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes < 8 AND J.porcentagemInstalacoes > 1.9;
UPDATE Caso02 SET nomeRange = 'Medium Facilities Percentage' WHERE porcentagemIMedia < 8 AND porcentagemIMedia > 1.9;

INSERT INTO Caso02 (porcentagemDMedia, porcentagemIMedia, IDH)
SELECT ROUND(AVG(J.porcentagemHomens + J.porcentagemMulheres), 4), ROUND(AVG(J.porcentagemInstalacoes), 4), ROUND(AVG(J.idh), 4)
FROM Juncao2 J WHERE J.porcentagemInstalacoes < 1.9;
UPDATE Caso02 SET nomeRange = 'Low Facilities Percentage' WHERE porcentagemIMedia < 1.9;

org.h2.jdbc.JdbcSQLException:  Column "IDH" not found; SQL statement